# The Official AVM FRITZ!Box Interfaces

AVM offers developers two ways to communicate with their FRITZ!Box routers:
* **TR-064:** 
* **AHA-HTTP:** 

The `fritzconnection` library uses both of these interfaces, at least according to its documentation. I'd like to understand how this communication works. 

First things first, I need to get the login info.

In [73]:
import json

# load config file
CONFIG_FILE = "..\\..\\..\\_private_files\\sb4dfritz_secrets.ini"
with open(CONFIG_FILE,"r") as file:
    FRITZBOX = json.load(file)
# extract login data
FRITZ_IP = FRITZBOX['login']['ip']
FRITZ_USER = FRITZBOX['login']['user']
FRITZ_PWD = FRITZBOX['login']['pwd']
# extract device ains
DEVICES = FRITZBOX['ains']
AINS = list(DEVICES.values())


## The TR-064 Method


Here's a basic way to get the "current" power consumption of a FRITZ! smart plug. 
The procedure goes as follows:
* Formulate the request as a SOAP action
* Send it as a HTTP POST message using the `requests` module

### A concrete example


### What actions are there?


### Implemententing the actions


#### GetSpecificDeviceInfos [WORKS]

In [4]:
import requests, warnings
from requests.auth import HTTPDigestAuth


def get_specific_device_info(device_ain:str)->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "GetSpecificDeviceInfos"
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                    <NewAIN>{device_ain}</NewAIN> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = get_specific_device_info(AINS[0])
print(results.headers)
print(results.text)

{'Connection': 'keep-alive', 'Content-Length': '1676', 'Content-Type': 'text/xml; charset="utf-8"', 'Date': 'Mon, 18 Aug 2025 18:03:48 GMT', 'Server': 'FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:GetSpecificDeviceInfosResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1">
<NewDeviceId>16</NewDeviceId>
<NewFunctionBitMask>35712</NewFunctionBitMask>
<NewFirmwareVersion>04.27</NewFirmwareVersion>
<NewManufacturer>AVM</NewManufacturer>
<NewProductName>FRITZ!DECT 200</NewProductName>
<NewDeviceName>l&apos;angolo del caffè</NewDeviceName>
<NewPresent>CONNECTED</NewPresent>
<NewMultimeterIsEnabled>ENABLED</NewMultimeterIsEnabled>
<NewMultimeterIsValid>VALID</NewMultimeterIsValid>
<NewMultimeterPower>0</NewMultimeterPower>
<NewMultimeterEnergy>2907513</NewMultimeterEnergy>
<NewTemperatureIsEnabled>ENABLED<


#### GetInfo [Works]

In [5]:
import requests, warnings
from requests.auth import HTTPDigestAuth

def get_info()->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "GetInfo"
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = get_info()
print(results.headers)
print(results.text)

{'Connection': 'keep-alive', 'Content-Length': '518', 'Content-Type': 'text/xml; charset="utf-8"', 'Date': 'Mon, 18 Aug 2025 18:03:49 GMT', 'Server': 'FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:GetInfoResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1">
<NewAllowedCharsAIN>0123456789ABCDEFabcdef :-grptmp</NewAllowedCharsAIN>
<NewMaxCharsAIN>19</NewMaxCharsAIN>
<NewMinCharsAIN>1</NewMinCharsAIN>
<NewMaxCharsDeviceName>79</NewMaxCharsDeviceName>
<NewMinCharsDeviceName>1</NewMinCharsDeviceName>
</u:GetInfoResponse>
</s:Body>
</s:Envelope>


#### GetGenericDeviceInfos [FAIL]

In [6]:
import requests, warnings
from requests.auth import HTTPDigestAuth

def get_generic_device_infos(device_index:str)->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "GetGenericDeviceInfos"
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                    <NewIndex>{device_index}</NewIndex> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = get_generic_device_infos("16")
print(results.headers)
print(results.text)

{'Connection': 'keep-alive', 'Content-Length': '441', 'Content-Type': 'text/xml; charset="utf-8"', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<s:Fault>
<faultcode>s:Client</faultcode>
<faultstring>UPnPError</faultstring>
<detail>
<UPnPError xmlns="urn:dslforum-org:control-1-0">
<errorCode>713</errorCode>
<errorDescription>SpecifiedArrayIndexInvalid</errorDescription>
</UPnPError>
</detail>
</s:Fault>
</s:Body>
</s:Envelope>


#### SetSwitch


In [7]:
import requests, warnings
from requests.auth import HTTPDigestAuth


def set_switch(device_ain:str, target_state:str)->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "SetSwitch"

    # ALLOWED_STATES = ["ON", "OFF", "TOGGLE"]
    # if not target_state in ALLOWED_STATES:
    #     print("Target state must be 'ON', 'OFF', or 'TOGGLE'.")
    #     return
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                    <NewAIN>{device_ain}</NewAIN> 
                    <NewSwitchState>{target_state}</NewSwitchState> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = set_switch(AINS[0], "ON")
print(results.headers)
print(results.text)


{'Connection': 'keep-alive', 'Content-Length': '278', 'Content-Type': 'text/xml; charset="utf-8"', 'Date': 'Mon, 18 Aug 2025 18:03:50 GMT', 'Server': 'FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:SetSwitchResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1"></u:SetSwitchResponse>
</s:Body>
</s:Envelope>


### Another go at `turn_off_when_idle`


#### Get current power

In [8]:
from datetime import datetime, timezone

def get_current_power(ain):
    # request info from device keeping track of start and end times
    start = current_time()
    device_info = get_specific_device_info(ain)
    end = current_time()
    # extract power (in W) and timestamp 
    power = int((device_info.text).split("<NewMultimeterPower>")[1].split("</NewMultimeterPower>")[0]) / 100
    timestamp = parse_date_str(device_info.headers['Date'])
    # compute duration and latency
    duration = seconds_between(start, end)
    latency = seconds_between(timestamp, end)
    power_data = {
        "power":power,
        "time":timestamp,
        "start":start,
        "end":end,
        "duration":duration,
        "latency":latency,
    }
    return power_data

def current_time():
    return datetime.now()#.astimezone()

def seconds_between(start:datetime, end:datetime)->float:
    return (end - start).total_seconds()

def parse_date_str(date_str):
    date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S GMT")
    date = date.replace(tzinfo=timezone.utc)
    date = date.astimezone().replace(tzinfo=None)
    return date

get_current_power(AINS[0])

{'power': 0.0,
 'time': datetime.datetime(2025, 8, 18, 20, 3, 51),
 'start': datetime.datetime(2025, 8, 18, 20, 3, 51, 335802),
 'end': datetime.datetime(2025, 8, 18, 20, 3, 52, 328075),
 'duration': 0.992273,
 'latency': 1.328075}

#### Detect power measurement cycle

Here's a first attempt to detect the power measurement cycle.
I'm simply requesting measurements repeatedly until the power value changes.

In [9]:
def detect_power_cycle(ain):
    # measure power once
    current_measurement = get_current_power(ain)
    # keep measuring until the power value changes
    last_measurement = current_measurement
    while last_measurement['power'] == current_measurement['power']:
        current_measurement = get_current_power(ain)
        print(current_measurement)
    return current_measurement['start']
    
detect_power_cycle(AINS[1])

{'power': 108.36, 'time': datetime.datetime(2025, 8, 18, 20, 3, 53), 'start': datetime.datetime(2025, 8, 18, 20, 3, 53, 322753), 'end': datetime.datetime(2025, 8, 18, 20, 3, 54, 247272), 'duration': 0.924519, 'latency': 1.247272}
{'power': 108.36, 'time': datetime.datetime(2025, 8, 18, 20, 3, 54), 'start': datetime.datetime(2025, 8, 18, 20, 3, 54, 249282), 'end': datetime.datetime(2025, 8, 18, 20, 3, 55, 240210), 'duration': 0.990928, 'latency': 1.24021}
{'power': 108.36, 'time': datetime.datetime(2025, 8, 18, 20, 3, 55), 'start': datetime.datetime(2025, 8, 18, 20, 3, 55, 240981), 'end': datetime.datetime(2025, 8, 18, 20, 3, 56, 177631), 'duration': 0.93665, 'latency': 1.177631}
{'power': 108.36, 'time': datetime.datetime(2025, 8, 18, 20, 3, 56), 'start': datetime.datetime(2025, 8, 18, 20, 3, 56, 179559), 'end': datetime.datetime(2025, 8, 18, 20, 3, 57, 122153), 'duration': 0.942594, 'latency': 1.122153}
{'power': 108.36, 'time': datetime.datetime(2025, 8, 18, 20, 3, 57), 'start': date

datetime.datetime(2025, 8, 18, 20, 4, 7, 724269)

Unfortunately, this doesn't behave as expected. The code works, but it takes too long to run (often longer than a minute, so far not longer than two). I'm suspecting that this is related to my previous observation that the smart plug have a sleep mode in which they're only sending power data every 2 minutes. Maybe the TR-064 request doesn't wake up the smart plug?

Let me try to confirm this.

In [10]:
ain = AINS[0]

power_jumps = []
# measure power once
current_measurement = get_current_power(ain)
while len(power_jumps) < 2:
    # print the measurement
    print(current_measurement)
    # keep measuring until the power value jumps
    last_measurement = current_measurement
    while last_measurement['power'] == current_measurement['power']:
        current_measurement = get_current_power(ain)
    # record the measurement after the jump
    power_jumps.append(current_measurement)
    # stop after two jumps
# compute time between jumps
jump1, jump2 = tuple([jump['time'] for jump in power_jumps])
seconds_between(jump1, jump2)


{'power': 1328.51, 'time': datetime.datetime(2025, 8, 18, 20, 4, 9), 'start': datetime.datetime(2025, 8, 18, 20, 4, 8, 800690), 'end': datetime.datetime(2025, 8, 18, 20, 4, 9, 874866), 'duration': 1.074176, 'latency': 0.874866}
{'power': 1302.4, 'time': datetime.datetime(2025, 8, 18, 20, 4, 46), 'start': datetime.datetime(2025, 8, 18, 20, 4, 46, 256770), 'end': datetime.datetime(2025, 8, 18, 20, 4, 47, 195727), 'duration': 0.938957, 'latency': 1.195727}


31.0

Yeah, that's about what I expected. There's roughly a 2 minute break (~120 seconds) between two power value jumps. So I'm confident to make the following conjectures:
* The FRITZ! smart plugs do have a sleep mode, in which they only send data every 120 seconds.
* The TR-064 request does not wake them up.
* The TR-064 request probably only interacts with the FRITZ!Box, and not with the smart plug.

### Random Notes
Note that this function returns the response as an instance of the `requests.Response` class.
While I'm at it, I might just as well take a quick look at what this class does.

In [11]:
# print documentation of the `requests.Response` class
print(help(results))

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |
 |  Methods defined here:
 |
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |
 |  __enter__(self)
 |
 |  __exit__(self, *args)
 |
 |  __getstate__(self)
 |      Helper for pickle.
 |
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |
 |      This attribute checks if 

The relevant information is found in the attributes `.headers` and `.text`. 
* `.headers` is a type of dictionary containing metadata of the response including a time stamp with second precision
* `.text` is a string containing the XML encoded response

In [12]:
# print data types
print("Type of .headers:", type(results.headers))
print("Type of .text:   ", type(results.text))
print("-"*80)
# print dictionary keys
print("Keys in results.headers:")
for key, val in results.headers.items():
    print(f"- {key:14s} : {val}")


Type of .headers: <class 'requests.structures.CaseInsensitiveDict'>
Type of .text:    <class 'str'>
--------------------------------------------------------------------------------
Keys in results.headers:
- Connection     : keep-alive
- Content-Length : 278
- Content-Type   : text/xml; charset="utf-8"
- Date           : Mon, 18 Aug 2025 18:03:50 GMT
- Server         : FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60
- Ext            : 


In [13]:
print("Type of .text:   ", type(results.text))
print("-"*80)
# print results.text
print("Contents of results.text:\n")
print(results.text)

Type of .text:    <class 'str'>
--------------------------------------------------------------------------------
Contents of results.text:

<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:SetSwitchResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1"></u:SetSwitchResponse>
</s:Body>
</s:Envelope>


In [14]:

def get_power(device_ain):
    """Get the current power consumption using the TR-064 interfaces."""
    # get specific device information
    request_result = get_specific_device_info(device_ain)
    # extract power value and date
    power = int((request_result.text).split("<NewMultimeterPower>")[1].split("</NewMultimeterPower>")[0]) / 100
    date = request_result.headers['Date']
    return power, date
    # return request_result

for dev, ain in DEVICES.items():
    print(dev, get_power(ain))

angolo (1313.13, 'Mon, 18 Aug 2025 18:05:18 GMT')
tv (79.96, 'Mon, 18 Aug 2025 18:05:18 GMT')


## The AHA-HTTP Method

### Sending Requests (assuming an SID is available) [WORK IN PROGRESS]

In [75]:
import requests

# Replace these with your values
sid = get_sid(FRITZ_USER, FRITZ_PWD)      # You must get this via login first


In [ ]:
# FRITZ!Box URL
# url = 'https://' + FRITZ_IP + '/webservices/homeautoswitch.lua'
url = 'http://fritz.box/webservices/homeautoswitch.lua'

ain = AINS[0].replace(" ", "")              # AIN of your smart device
# cmd = 'getswitchlist'               # Example command (see command list below)
cmd = 'getswitchpower'               # Example command (see command list below)
cmd = 'getbasicdevicestats'               # Example command (see command list below)
cmd = 'getswitchpower'               # Example command (see command list below)

# Parameters for the GET request
params = {
    'ain': ain.replace("", " "),
    # 'ain': ain,
    'switchcmd': cmd,
    'sid': sid
}

get_request_url = f"{url}?ain={ain}&switchcmd={cmd}&sid={sid}"
print(get_request_url)
# Send the GET request
# response = requests.get(get_request_url)  # Use verify=False if self-signed cert
response = requests.get(get_request_url, verify=False)  # Use verify=False if self-signed cert
# response = requests.get(url, params=params, verify=False)  # Use verify=False if self-signed cert
print(response)
print(response.text)

http://fritz.box/webservices/homeautoswitch.lua?ain=087610406876&switchcmd=getbasicdevicestats&sid=ab08ffc558308a86
<Response [200]>
<devicestats><temperature><stats count="96" grid="900" datatime="1755548546">240,255,260,270,260,265,250,250,265,250,250,255,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265</stats></temperature><voltage><stats count="360" grid="10" datatime="1755548546">231338,231338,231338,230690,230690,230690,230690,230917,231361,231150,231714,231358,231597,228433,231118,229935,231408,231774,231569,231569,231569,230775,230775,230775,230775,230775,230775,230775,230775,230775,230775,230775,230775,231155,231155,231155,231155,231155,231155,231155,231155,231155,231155,231155,231155,230524,230

In [72]:
params = {
    # 'ain': ain.replace("", " "),
    'ain': ain,
    'switchcmd': cmd,
    'sid': sid
}

params = [f"{key}={val}" for key, val in params.items()]
print(params)
params = "&".join(params)
print(params)
params = params.replace(" ", "")
print(params)
url+"?"+params

['ain=087610406876', 'switchcmd=getbasicdevicestats', 'sid=ab08ffc558308a86']
ain=087610406876&switchcmd=getbasicdevicestats&sid=ab08ffc558308a86
ain=087610406876&switchcmd=getbasicdevicestats&sid=ab08ffc558308a86


'http://fritz.box/webservices/homeautoswitch.lua?ain=087610406876&switchcmd=getbasicdevicestats&sid=ab08ffc558308a86'

```xml
<devicestats>
    <temperature>
        <stats count="96" grid="900" datatime="1755547537">
            255,260,270,260,265,250,250,265,250,250,255,265,
            265,265,265,265,265,265,265,265,265,265,265,265,
            265,265,265,265,265,265,265,265,265,265,265,265,
            265,265,265,265,265,265,265,265,265,265,265,265,
            265,265,265,265,265,265,265,265,265,265,265,265,
            265,265,265,265,265,265,265,265,265,265,265,265,
            265,265,265,265,265,265,265,265,265,265,265,265,
            265,265,265,265,265,265,265,265,265,265,265,265
        </stats>
    </temperature>
    
    <voltage>
        <stats count="360" grid="10" datatime="1755547537">
            229375,229375,229375,229375,229375,229375,229375,229375,229375,229375,229375,229375,
            230597,230597,230597,230597,230597,230597,230597,230597,230597,230597,230597,230597,
            231582,231582,231582,231582,231582,231582,231582,231582,231582,231582,231582,231582,
            231959,231959,231959,231959,231959,231959,231959,231959,231959,231959,231959,231959,
            230755,230755,230755,230755,230755,230755,230755,230755,230755,230755,230755,230755,
            228498,228498,228498,228498,228498,228498,228498,228498,228498,228498,228498,228498,
            231426,231426,231426,231426,231426,231426,231426,231426,231426,231426,231426,231426,
            230711,230711,230711,230739,230739,230739,230676,230676,230676,231518,231518,231518,
            231518,231518,231518,231518,231386,231386,231386,230170,230170,230170,230551,230551,
            230551,228499,228499,228499,228499,228499,228090,227567,227567,227567,227567,228022,
            230594,231119,231119,231119,231119,231119,231119,231119,231119,231119,231119,231119,
            231119,231444,231444,231444,231444,231444,231444,231444,231444,231444,231444,231444,
            231444,231575,231575,231575,231575,231575,231575,231575,231575,231575,231575,231575,
            231575,231324,231324,231324,231324,231324,231324,231324,231324,231324,231324,231324,
            231324,230795,230795,230795,230795,230795,230795,230795,230795,230795,230795,230795,
            230795,230947,230947,230947,230947,230947,230947,230947,230947,230947,230947,230947,
            230947,230890,230890,230890,230890,230890,230890,230890,230890,230890,230890,230890,
            230890,230371,230371,230371,230371,230371,230371,230371,230371,230371,230371,230371,
            230371,229893,229893,229893,229893,229893,229893,229893,229893,229893,229893,229893,
            229893,230281,230281,230281,230281,230281,230281,230281,230281,230281,230281,230281,
            230281,229917,229917,229917,229917,229917,229917,229917,229917,229917,229917,229917,
            229917,230461,230461,230461,230461,230461,230461,230461,230461,230461,230461,230461,
            230461,230009,230026,229851,229696,229751,229556,229634,229502,229884,229581,229709,
            229808,229822,229802,227352,229103,229698,228182,229195,229400,229507,229107,229107,
            229275,229275,229275,229275,229275,229275,229275,229275,226563,226563,226563,226563,
            226563,226563,226563,226563,227200,229840,229840,229840,229840,229840,229840,229840,
            229840,229840,229840,229955,229955,229955,229955,229955,229955,229955,229955,229955,
            229955,229955,229955,231223,231223,231223,231223,231223,231223,231223,231223,231223,
            231223,231223,231223,231241,231241,231241,231241,231241,231241,231241,231241,231241,
            231241,231241,231241,232315,232315,232315,232315,232315,232315,232315,232315,232315
        </stats>
    </voltage>
    
    <power>
        <stats count="360" grid="10" datatime="1755547537">
            329,329,329,329,329,329,329,329,329,329,329,329,
            329,329,329,329,329,329,329,329,329,329,329,329,
            329,329,329,329,329,329,329,329,329,329,329,329,
            329,329,329,329,329,329,329,329,329,329,329,329,
            329,329,329,329,329,329,329,329,329,329,329,329,
            715,715,715,715,715,715,715,715,715,715,715,715,
            43637,43637,43637,43637,43637,43637,43637,43637,
            43637,43637,43637,43637,336,336,336,105779,105779,
            105779,321,321,321,95086,95086,95086,95086,95086,
            95086,95086,357,357,357,100393,100393,100393,38001,
            38001,38001,133252,133252,133252,133252,133252,132179,
            132007,132007,132007,132007,133059,207,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,321,329,93591,
            329,329,81532,321,329,357,17952,17952,350,350,350,350,
            350,350,350,350,130577,130577,130577,130577,130577,
            130577,130577,130577,132000,207,207,207,207,207,207,
            207,207,207,207,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
        </stats>
    </power>
    
    <energy>
        <stats count="12" grid="2678400" datatime="1755537790">
            20566,32426,30118,34865,26693,31961,31611,37307,34098,33329,33660,29178
        </stats>
        <stats count="31" grid="86400" datatime="1755537790">
            808,0,0,431,1319,1216,1280,1448,980,1475,1337,1467,1343,1259,1548,1259,
            1581,1815,1467,1369,1088,1324,1533,1921,1149,1306,1508,0,0,0,0
        </stats>
    </energy>
</devicestats>
```

### Obtaining an SID (aka The Login Torture)

The login procedure is described [in the official documentation](https://fritz.com/fileadmin/user_upload/Global/Service/Schnittstellen/AVM_Technical_Note_-_Session_ID_english_2021-05-03.pdf). The bad news is that it's not entirely straight forward. However, the good news is that the documentation actually provides Python code. Here is a minor modification. The main feature is the function `get_sid()`.

In [74]:
#!/usr/bin/env python3
# vim: expandtab sw=4 ts=4
"""
FRITZ!OS WebGUI Login
Get a sid (session ID) via PBKDF2 based challenge response algorithm.
Fallback to MD5 if FRITZ!OS has no PBKDF2 support.
AVM 2020-09-25
"""

import sys
import hashlib
import time
import urllib.request
import urllib.parse
import xml.etree.ElementTree as ET

LOGIN_SID_ROUTE = "/login_sid.lua?version=2"


class LoginState:
    def __init__(self, challenge: str, blocktime: int):
        self.challenge = challenge
        self.blocktime = blocktime
        self.is_pbkdf2 = challenge.startswith("2$")


def get_sid(username: str, password: str, address:str="fritz.box") -> str:
    """ Get a sid by solving the PBKDF2 (or MD5) challenge-response
    process. """
    box_url = "http://" + address
    try:
        state = get_login_state(box_url)
    except Exception as ex:
        raise Exception("failed to get challenge") from ex
    if state.is_pbkdf2:
        # print("PBKDF2 supported")
        challenge_response = calculate_pbkdf2_response(state.challenge, password)
    else:
        # print("Falling back to MD5")
        challenge_response = calculate_md5_response(state.challenge, password)
    if state.blocktime > 0:
        # print(f"Waiting for {state.blocktime} seconds...")
        time.sleep(state.blocktime)
    try:
        sid = send_response(box_url, username, challenge_response)
    except Exception as ex:
        raise Exception("failed to login") from ex
    if sid == "0000000000000000":
        raise Exception("wrong username or password")
    return sid


def get_login_state(box_url: str) -> LoginState:
    """ Get login state from FRITZ!Box using login_sid.lua?version=2 """
    url = box_url + LOGIN_SID_ROUTE
    http_response = urllib.request.urlopen(url)
    xml = ET.fromstring(http_response.read())
    # print(f"xml: {xml}")
    challenge = xml.find("Challenge").text
    blocktime = int(xml.find("BlockTime").text)
    return LoginState(challenge, blocktime)


def calculate_pbkdf2_response(challenge: str, password: str) -> str:
    """ Calculate the response for a given challenge via PBKDF2 """
    challenge_parts = challenge.split("$")
    # Extract all necessary values encoded into the challenge
    iter1 = int(challenge_parts[1])
    salt1 = bytes.fromhex(challenge_parts[2])
    iter2 = int(challenge_parts[3])
    salt2 = bytes.fromhex(challenge_parts[4])
    # Hash twice, once with static salt...
    hash1 = hashlib.pbkdf2_hmac("sha256", password.encode(), salt1, iter1)
    # Once with dynamic salt.
    hash2 = hashlib.pbkdf2_hmac("sha256", hash1, salt2, iter2)
    return f"{challenge_parts[4]}${hash2.hex()}"


def calculate_md5_response(challenge: str, password: str) -> str:
    """ Calculate the response for a challenge using legacy MD5 """
    response = challenge + "-" + password
    # the legacy response needs utf_16_le encoding
    response = response.encode("utf_16_le")
    md5_sum = hashlib.md5()
    md5_sum.update(response)
    response = challenge + "-" + md5_sum.hexdigest()
    return response


def send_response(box_url: str, username: str, challenge_response: str)->str:
    """ Send the response and return the parsed sid. raises an Exception on
    error """
    # Build response params
    post_data_dict = {"username": username, "response": challenge_response}
    post_data = urllib.parse.urlencode(post_data_dict).encode()
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    url = box_url + LOGIN_SID_ROUTE
    # Send response
    http_request = urllib.request.Request(url, post_data, headers)
    http_response = urllib.request.urlopen(http_request)
    # Parse SID from resulting XML.
    xml = ET.fromstring(http_response.read())
    return xml.find("SID").text


sid = get_sid(FRITZ_USER, FRITZ_PWD)
print("Successful login.")
print(f"SID: {sid}")


Successful login.
SID: dc4e0b93219e33a0
